# Satellite Imagery from Sentinel 2

Before running the notebook, do the following steps first (following instructions and code [here](https://github.com/artefactory/medium_satellite_imagery)):

*   create a geojson file of the zone you want a satellite imagery of and save it in 1_collection/city.geojson
*   create an account at Coppernicus Open Access Hub. Write the username and the password in a json file and save it in 1_collection/sentinel_api_credentials.json. Format: {"username": "XXXX", "password": "XXXX"}




## Step 0: Requirements & Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rasterio
!pip install sentinelsat

In [ ]:
import os 
import cv2
from datetime import date
import json
import itertools
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import pandas as pd

## Step 1: Authentication with Copernicus Open Access Hub

In [ ]:
with open("./sentinel_api_credentials.json", "r") as infile:
    credentials = json.load(infile)

api = SentinelAPI(credentials["username"], credentials["password"], "https://scihub.copernicus.eu/dhus")

## Step 2: Define area of interest in a geojson file

A way to specify the zones you want to explore is to provide the API with a geojson file, which is a json containing GPS coordinates position (latitude and longitude) of a zone. Geojson can be quickly created on this [website](https://geojson.io/#map=2/20/0).

In [ ]:
geojson_path = "./dnipro.geojson"

## Step 3: Query satellite images via Sentinelsat API

In [ ]:
shape = geojson_to_wkt(read_geojson(geojson_path))

images = api.query(
    shape,
    date=(date(2021, 7, 1), date(2022, 1, 30)),
    platformname="Sentinel-2",
    processinglevel = "Level-2A",
    cloudcoverpercentage=(0, 5)
)

images_df = api.to_dataframe(images)

We can use the is_online() function from the sentinelsat package to filter the queried images that are available online. A for loop iterates over the images dictionary and checks if each image is online. 

In [ ]:
online_images = {}
for id, item in images.items():
    if api.is_online(id):
        online_images[id] = item

# Convert the filtered images to a dataframe
online_images_df = api.to_dataframe(online_images)

In [ ]:
key_columns = ["platformserialidentifier", "processinglevel", "cloudcoverpercentage", 
               "vegetationpercentage", "beginposition"]
online_images_df = online_images_df.sort_values(by=['cloudcoverpercentage'], ascending=True)
online_images_df[key_columns].head(5)

,platformserialidentifier,processinglevel,cloudcoverpercentage,vegetationpercentage,beginposition
cfc78747-a2c3-4a82-bf4b-bd8665d16fac,Sentinel-2B,Level-2A,0.017637,34.068552,2021-10-01 08:47:19.024
3e2726a9-4abe-4900-bd04-64ce542bcae3,Sentinel-2B,Level-2A,0.018942,45.297468,2021-09-08 08:35:59.024
b53c275f-5f78-4b11-8223-e2c17ae1a9f8,Sentinel-2A,Level-2A,0.023305,12.639606,2021-10-26 08:50:41.024
615dc384-9606-4615-8071-be0337f90aee,Sentinel-2A,Level-2A,0.040223,71.198428,2021-07-08 08:46:01.024
2912ebbe-9e0f-4fbb-a516-1e7b7a55489a,Sentinel-2B,Level-2A,0.089014,68.727303,2021-07-20 08:35:59.024


##  Step 4: Download satellite image

In [ ]:
uuid = "2912ebbe-9e0f-4fbb-a516-1e7b7a55489a"
api.download(uuid)

MD5 checksumming:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

{'id': '2912ebbe-9e0f-4fbb-a516-1e7b7a55489a',
 'title': 'S2B_MSIL2A_20210720T083559_N0301_R064_T36UXU_20210720T113645',
 'size': 1130469573,
 'md5': 'a89da5754f9362c4b86980763c9d4987',
 'date': datetime.datetime(2021, 7, 20, 8, 35, 59, 24000),
 'footprint': 'POLYGON((34.36027924628367 48.74498411169783,35.85243870847307 48.71769475117851,35.79809471530292 47.73104447774509,34.334331348966806 47.75740973734738,34.36027924628367 48.74498411169783))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('2912ebbe-9e0f-4fbb-a516-1e7b7a55489a')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2021, 7, 20, 15, 10, 6, 480000),
 'Ingestion Date': datetime.datetime(2021, 7, 20, 15, 9, 32, 798000),
 'manifest_name': 'manifest.safe',
 'product_root_dir': 'S2B_MSIL2A_20210720T083559_N0301_R064_T36UXU_20210720T113645.SAFE',
 'quicklook_url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('2912ebbe-9e0f-4fbb-a516-1e7b7a55489a')/Products('Quicklook')/$value",
 'path': 'S2B

## Step 5: Unzip satellite image and save to folder

In [ ]:
!unzip /content/S2B_MSIL2A_20210720T083559_N0301_R064_T36UXU_20210720T113645.zip -d ../data/sentinel2/dnipro

# Credits

*   Antoine Aubay (2021): Leveraging satellite imagery for machine learning computer vision applications. Available at: https://medium.com/artefact-engineering-and-data-science/leveraging-satellite-imagery-for-machine-learning-computer-vision-applications-d22143f72d94

